In [0]:
# needed to import excel files
%pip install -i https://pypi.org/simple openpyxl

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
from datetime import datetime, timedelta 
from getpass import getuser as get_user 
from getpass import getpass as get_password 
from os import environ as enviroment_vars 
from os import setpgrp as set_process_group 
from random import random 
from shlex import split as cmd_split 
from string import punctuation 
from subprocess import Popen, PIPE, STDOUT 
import pandas as pd 


In [0]:
from pyspark import SparkContext, HiveContext, SparkConf  
from pyspark.sql import SparkSession 

ss = SparkSession(sc) 
ss 

In [0]:
# Create datetime filters for the monthly calculation
from dateutil.relativedelta import relativedelta

today = datetime.today()

if today.month == 1:
    month = 12
    year = today.year - 1
else:
    month = today.month - 1
    year = today.year

CURRENT_YTD_END = datetime(year, month, 1) + relativedelta(day=31)
CURRENT_YTD_BEG = datetime(year, 1, 1)

if today.month == 1:
    month = 12
    year = today.year - 2
else:
    month = today.month - 1
    year = today.year - 1

PRIOR_YTD_END = datetime(year, month, 1) + relativedelta(day=31)
PRIOR_YTD_BEG = datetime(PRIOR_YTD_END.year, 1, 1)

REPORT_MNTH_6 = datetime(today.year, today.month, 29) + relativedelta(months=-7)
REPORT_MNTH_18 = datetime(today.year, today.month, 1) + relativedelta(months=-18)
REPORT_MNTH_19 = datetime(today.year, today.month, 29) + relativedelta(months=-19)
REPORT_MNTH_30 = datetime(today.year, today.month, 1) + relativedelta(months=-30)
PRIOR_YTD_END2 = datetime(today.year, today.month, 1) + relativedelta(years=-2, day=31)
PRIOR_YTD_BEG2 = datetime(today.year, today.month, 1) + relativedelta(years=-3)

# ONLY USE BELOW in Jan each year
# CURRENT_YTD_END = datetime(today.year-1, 12, 31) #change date number to last day of the previous month
# CURRENT_YTD_BEG = datetime(today.year-1, 1, 1)
# PRIOR_YTD_END = datetime(today.year - 2, 12, 31) #change date number to last day of the previous month
# PRIOR_YTD_BEG = datetime(PRIOR_YTD_END.year, 1, 1)
# REPORT_MNTH_6 = datetime(today.year, today.month, 31) + relativedelta(months=-7)
# REPORT_MNTH_18 = datetime(today.year, today.month, 1) + relativedelta(months=-18)
# REPORT_MNTH_19 = datetime(today.year, today.month, 31) + relativedelta(months=-19)
# REPORT_MNTH_30 = datetime(today.year, today.month, 1) + relativedelta(months=-30)
# PRIOR_YTD_END2 = datetime(today.year, today.month, 31) + relativedelta(months=-25) #change date number to last day of the previous month
# PRIOR_YTD_BEG2 = datetime(today.year, 1, 1) + relativedelta(months=-36)

print("Current year end: " + str(CURRENT_YTD_END))
print("Current year beginning: " + str(CURRENT_YTD_BEG))
print("Previous year end: " + str(PRIOR_YTD_END))
print("Previous year beginning: " + str(PRIOR_YTD_BEG))
print("REPORT_MNTH_6: " + str(REPORT_MNTH_6))
print("REPORT_MNTH_18: " + str(REPORT_MNTH_18))
print("PRIOR_YTD_END2: " + str(PRIOR_YTD_END2))
print("PRIOR_YTD_BEG2: " + str(PRIOR_YTD_BEG2))

Current year end: 2024-08-31 00:00:00
Current year beginning: 2024-01-01 00:00:00
Previous year end: 2023-08-31 00:00:00
Previous year beginning: 2023-01-01 00:00:00
REPORT_MNTH_6: 2024-02-29 00:00:00
REPORT_MNTH_18: 2023-03-01 00:00:00
PRIOR_YTD_END2: 2022-09-30 00:00:00
PRIOR_YTD_BEG2: 2021-09-01 00:00:00


In [0]:
# pulling ecoplus loans
eco_plus = spark.sql(
    """ 
with eco as (
SELECT 
DISTINCt cmhcno,
    CASE
        WHEN UPPER(rurmktxt) LIKE '%++EP++%' THEN 1
        ELSE 0
    END AS eco_plus_app_ind,
    CASE
        WHEN UPPER(rurmktxt) LIKE '%++EPCAN++%' THEN 1
        ELSE 0
    END AS eco_plus_canc_ind
FROM dbenp001.tben_eventlog
WHERE (UPPER(rurmktxt) LIKE '%++EP++%'
OR UPPER(rurmktxt) LIKE '%++EPCAN++%')
AND SUBSTR(apidstmp, 1, 4) >= '2016'
)
select

cmhcno,
case when sum(eco_plus_app_ind)>0 then 1 else 0 end as eco_plus_app_ind,
case when sum(eco_plus_canc_ind)>0 then 1 else 0 end as eco_plus_canc_ind

from eco
group by cmhcno;
"""
)
eco_plus.createOrReplaceTempView("eco_plus")

In [0]:
# Calculate Loan level Base Data for previous and current YTD
# Pull in 2 years HO High Ratio ALL CMHC Purchase and Refinance loans
a = {
    "CURRENT_YTD_BEG": CURRENT_YTD_BEG,
    "CURRENT_YTD_END": CURRENT_YTD_END,
    "PRIOR_YTD_BEG": PRIOR_YTD_BEG,
    "PRIOR_YTD_END": PRIOR_YTD_END,
    "REPORT_MNTH_6": REPORT_MNTH_6,
    "REPORT_MNTH_18": REPORT_MNTH_18,
    "PRIOR_YTD_BEG2": PRIOR_YTD_BEG2,
    "PRIOR_YTD_END2": PRIOR_YTD_END2,
}

sql = """
select distinct
    lad.cmhc_loan_account_nbr,
    emili.aplcrecd, 
    emili.aplcapro,
    lad.loan_purpose_type_desc_en,
    lad.loan_purpose_tcd,
Case when emili.ACSUCD = 0 then 'ENTRD'
     when emili.ACSUCD = 1 then 'RECEVD'
     when emili.ACSUCD = 2  then 'CANCEL'
     when emili.ACSUCD = 3 then 'REJECT'
     when emili.ACSUCD = 4 then 'REFUSE'
     when emili.ACSUCD = 5  then 'WITHDN'
     when emili.ACSUCD = 6 then 'APPRVD'
     when emili.ACSUCD = 7  then 'ISSUED'
     when emili.ACSUCD = 8  then 'RISKED'
     when emili.ACSUCD = 9  then 'FINAL'   
     else ' ' end AS LOAN_STATUS_EN,
case when lad.loan_purpose_tcd in ('1', '4', '7', '8', '9', 'D', '**') then 'Purchase'       
     when lad.loan_purpose_tcd in ('2', '3', '5', 'A', 'E') then 'Refinance'
     else 'Unknown' end as loan_purpose,
    las.property_lending_value_amt AS LENDING_VALUE,
    las.loan_insured_amt AS INSUR_AMT,
    las.loan_lender_amt AS LOAN_AMT,
    emili.ACSUCD,
    emili.TOTUTCT as Unit,
    lad.low_ratio_mortgage_tcd,



CASE WHEN emili.aplcapro is null THEN 'NEVER APPROVED' 
    ELSE 'EVER APPROVED' END AS EVER_APPROVED,

case when las.ever_approved_date_surrkey<>3 then "Ever_Approved" else "Never Aprroved" end as EVER_APPROVED2, /*Added 31.07.23*/


case when ( emili.aplcrecd between '{0}' and '{1}') and (emili.aplcapro between '{0}' and '{1}') 
    then 1 else 0
    end as Approved_YTD,

case when ( emili.aplcrecd between '{0}' and '{1}') and ((emili.aplcapro between '{0}' and '{1}') or 
(las.finalized_date_surrkey between '{0}' and '{1}') or (emili.APLREJDT between '{0}' and '{1}')) 
THEN 1 ELSE 0
END AS RISKED_YTD,

case when ( emili.aplcrecd between '{2}' and '{3}') and (emili.aplcapro between '{2}' and '{3}') 
then 1 else 0
end as Approved_PYTD,

case when ( emili.aplcrecd between '{2}' and '{3}')and ((emili.aplcapro between '{2}' and '{3}') or 
(las.finalized_date_surrkey between '{2}' and '{3}') or (emili.APLREJDT between '{2}' and '{3}')) 
THEN 1 ELSE 0
END AS RISKED_PYTD,

/*CANCELLATION RATE*/
CASE WHEN ( emili.aplcrecd between '{5}' and '{4}') THEN 1 ELSE 0
END AS REC_CXL_YTD,

CASE WHEN ( emili.aplcrecd between '{5}' and '{4}') AND emili.ACSUCD = 2 
THEN 1 ELSE 0
END AS CANCELLED_YTD,

/* FUNDING RATE sum('main_data CDIH'[finalized])/sum('main_data CDIH'[approved_funded_period] */

CASE WHEN emili.ACSUCD = 9 THEN 1 ELSE 0 
END AS finalized_loan,

CASE
        WHEN (las.ever_approved_date_surrkey BETWEEN '{5}' AND '{4}' AND emili.ACSUCD in (9,6,7) AND emili.FPRMRCAM > 50 AND emili.totinsat > 0) THEN 1 ELSE 0
    END AS finalized, /* changed to ever approved 23.05.2024 ( changed from emili.aplcapro to risk_f.aplcapro), and into NBW */
    
emili.FPRMRCAM,
las.ever_approved_date_surrkey,

case 
    when (las.ever_approved_date_surrkey between '{5}' and '{4}') then 1 else 0
end as approved_funded_period,   /*  changed to ever approved (  changed from emili.aplcapro to risk_f.aplcapro) */

CASE WHEN  pad.urban_rural_cde in ('2','91') AND pad.current_metropolitan_major_area_cde IN ('9000') THEN 'RURAL'
    ELSE 'URBAN' 
    END AS urban_rural,

emili.FPRMRCDT as Fprem_date_rec,
emili.FPRMRCAM as Fprem_amount, 

CASE WHEN pad.current_stats_canada_province_nm_en IN ('NEWFOUNDLAND AND LABRADOR', 'NEW BRUNSWICK', 'PRINCE EDWARD ISLAND', 
    'NOVA SCOTIA') THEN 'ATLANTIC' 
    WHEN pad.current_stats_canada_province_nm_en IN ('ALBERTA', 'MANITOBA', 'SASKATCHEWAN', 'YUKON', 'NUNAVUT', 
    'NORTHWEST TERRITORIES') THEN 'PRAIRIES' 
    ELSE pad.current_stats_canada_province_nm_en 
END AS REGION, 
pad.current_metropolitan_major_area_nm as met_name, /*ADDEDxxxxxxxxxxxxxxxxxxxxxxxxxxxxx*/
CASE 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('2270') THEN 'Toronto' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0120') THEN 'Barrie' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0125') THEN 'Brantford' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('2000') THEN 'Greater Sudbury' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0460','4000') THEN 'Guelph' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0610') THEN 'Hamilton' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0700') THEN 'Kingston' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0850') THEN 'Kitchener-Cambridge-Waterloo' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('0950') THEN 'London' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('1250') THEN 'Oshawa' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('1320','4850') THEN 'Peterborough' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('1160') THEN 'St. Catharines-Niagara' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('2240') THEN 'Oshawa' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('2270') THEN 'Oshawa' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('2640') THEN 'Oshawa' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'ONTARIO' AND pad.current_metropolitan_major_area_cde IN ('1265') THEN 'Ottawa' 
    WHEN pad.current_stats_canada_province_nm_en  LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde IN ('1264') THEN 'Gatineau' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'ALBERTA' AND pad.current_metropolitan_major_area_cde LIKE '0140' THEN 'Calgary' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'ALBERTA' AND pad.current_metropolitan_major_area_cde LIKE '0340' THEN 'Edmonton' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'BRITISH COLUMBIA' AND pad.current_metropolitan_major_area_cde LIKE '2410' THEN 'Vancouver'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'BRITISH COLUMBIA' AND pad.current_metropolitan_major_area_cde LIKE '0110' THEN 'Abbotsford-Mission'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'BRITISH COLUMBIA' AND pad.current_metropolitan_major_area_cde LIKE '4240' THEN 'Kelowna'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'BRITISH COLUMBIA' AND pad.current_metropolitan_major_area_cde LIKE '2440' THEN 'Victoria'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'MANITOBA' AND pad.current_metropolitan_major_area_cde LIKE '2680' THEN 'Winnipeg'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'NEW BRUNSWICK' AND pad.current_metropolitan_major_area_cde LIKE '4400' THEN 'Moncton'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'NEW BRUNSWICK' AND pad.current_metropolitan_major_area_cde LIKE '1600' THEN 'Saint John'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'Newfoundland and Labrador' AND pad.current_metropolitan_major_area_cde LIKE '1640' THEN 'St. Johns'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'Nova Scotia' AND pad.current_metropolitan_major_area_cde LIKE '0580' THEN 'Halifax'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'Saskatchewan' AND pad.current_metropolitan_major_area_cde LIKE '1490' THEN 'Regina'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'Saskatchewan' AND pad.current_metropolitan_major_area_cde LIKE '1700' THEN 'Saskatoon'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde LIKE '1060' THEN 'Montreal'
    WHEN pad.current_stats_canada_province_nm_en LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde LIKE '1400' THEN 'Quebec' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde LIKE '0180' THEN 'Saguenay' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde LIKE '1800' THEN 'Sherbrooke' 
    WHEN pad.current_stats_canada_province_nm_en LIKE 'QUEBEC' AND pad.current_metropolitan_major_area_cde LIKE '2320' THEN 'Trois-Rivieres' 
    WHEN pad.urban_rural_cde in ('2','91') AND pad.current_metropolitan_major_area_cde = '9000' THEN 'Rural'
    else 'Urban-Other CMA' 
    end as CMA_URRUR,

pad.current_stats_canada_division_cde,
pad.current_stats_canada_subdivision_cde,
pad.current_metropolitan_major_area_cde,

CASE WHEN pad.dwelling_tcd IN ('008','003','018') THEN 'PLEXES' 
    WHEN pad.dwelling_tcd IN ('005') THEN 'APPART.' 
    WHEN pad.dwelling_tcd IN ('002','010','004') THEN 'SEMI-DETACHED/STACKED/ROW' 
    WHEN pad.dwelling_tcd IN ('006') THEN 'MOBILE' 
    WHEN pad.dwelling_tcd IN ('017') THEN 'MODULAR' 
    when pad.dwelling_tcd IN ('001') THEN 'SINGLE'
    ELSE pad.dwelling_tcd
    END AS dwelling_cat,

emili.aplsincc as aplic_income_course,

pad.dwelling_type_desc_en as dwelling_type_name,
lad.loan_security_type_desc_en,

case
    when lad.loan_security_tcd in ('4') then 1 
    else 0 end as Ministerial_Guarantee_Ind,
lad.loan_security_tcd,
lad.interest_tcd,
lad.interest_type_desc_en as Interest_Type,

/*
case
    when lad.interest_tcd in ('1') then 'Fixed'
    when lad.interest_tcd in ('2') then 'Adjustable'
    when lad.interest_tcd in ('3') then 'Buydown'
    when lad.interest_tcd in ('4') then 'VRM'
    when lad.interest_tcd in ('5') then 'Indexed LK'
    when lad.interest_tcd in ('6') then 'Capped VRM'
    else '9' end as Interest_Type,
  */

trim(lad.business_source_identification_desc_en) as BUSINESS_SOURCE_EN,
trim(lad.application_source_type_desc_en) as Applic_SOURCE_EN,

concat(trim(lad.original_representative_first_nm), ' ', trim(lad.original_representative_last_nm)) as Original_Rep, /*Brench rep*/
trim(lad.contact_nm) as Contact_nm, /*lender's RU*/

lend.lender_cde as Lender_cde, /*"ADMLENDR Initiation lender code branch"*/
lend.lender_nm_en as Lender_EN, /*"Initiation lender en (branch)"	*/
lend.parent_level1_lender_nm_en, /*"Initiation Parent level 1 lender no"*/
lend.parent_level1_lender_cde,
lend.parent_level2_lender_nm_en, /*"Initiation Parent level 1 lender no"*/
lend.parent_level2_lender_cde,

lad.installment_completion_type_desc_en as progress_advance,
   
trim(emili.FINTSTNO) as TRANSIT,
concat(trim(emili.FININSCD), '-', trim(emili.FINTSTNO)) as financial_transit,
concat(trim(emili.FININSCD), trim(emili.FINTSTNO)) as Transit_DESJ,
concat(trim(lad.contact_local_telephone_area_cde), trim(lad.contact_local_telephone_nbr)) as LENDER_TEL_NO,
 
emili.GDSRATIO as GDS, 
emili.TDSRATIO AS TDS, 
las.loan_to_value_rtio as LTV_RATIO, 

(emili.TOTINSAT/emili.BRWTOTIN)*100 as LTI,
emili.BRWTOTIN as famili_income,


case when pad.new_existing_building_tcd = '1' then 'New'
    when pad.new_existing_building_tcd  = '2' then 'Existing'
    else 'NA'
    end as New_Exist, 
pad.new_existing_building_type_desc_en,

concat(trim(emili.TENURECD), '-', trim(pad.tenure_type_desc_en)) as Tenure_EN, 
concat(trim(pad.dwelling_tcd), '-', trim(pad.dwelling_type_desc_en)) as Dwelling_EN, 

case 
    when lad.occupancy_tcd is null then 'No Code'
    when lad.occupancy_tcd in ('01') then 'Owner Occupied'
    when lad.occupancy_tcd in ('02') then 'Rental/Investment'
    else 'N/A'
    end as Occupancy_EN,

pad.property_address_municipality_nm as PROP_Municipality,
emili.GFTEQTIN as Gift_dp,
Case when emili.GFTEQTIN = 1 then "Gifted DP"
    else "No Gift" end as Gifted_dp,

bor.borrower_age_year_cnt as FBOR_AGE, 

case when emili.cursstcd in ('02','03') then 'First Time' /*2=RENT, 3= LIVE WITH PARENTS*/ 
    else 'Repeat' end as HOMEBUYER_TYPE,

CASE WHEN emili.GENCRSC2 IS NULL THEN 'Single Borrower' 
    ELSE 'Multiple Borrowers' 
    END as single_multiple_borr,

bor.non_permanent_resident_ind as FBOR_NON_PERM_RESIDENT_IND,
Case when bor.non_permanent_resident_ind = 1 then "Non-perm resident"
    else "Perm resident" end as Perm_resident,

pad.current_stats_canada_province_nm_en as PROP_PROV_EN, 

case
    when lad.cmhc_loan_account_nbr is not null then 1 
    else 0
    end as Loan,

case
    when lad.business_source_identification_cde not in ('04', '13') then 'BRANCH' 
    else lad.business_source_identification_cde  
    end as Regroup_B_Source,
    

substr(cast(emili.aplcrecd as string),1,4) as rec_year,
substr(cast(emili.aplcrecd as string),5,2) as rec_month,
substr(cast(emili.aplcapro as string),1,4) as apr_year,
substr(cast(emili.aplcapro as string),5,2) as apr_month

/*
CASE WHEN month(aplcrecd) < 10 then CONCAT('0', month(aplcrecd))
else month(aplcrecd)
END AS MREC,
CASE WHEN month(aplcapro) < 10 then CONCAT('0', month(aplcapro))
else month(aplcapro)
END AS MEVER*/

FROM cmwdw_insurance.vwdw_loan_account_dim lad


Left join cmwdw_insurance.vwdw_loan_account_sum las
    ON lad.loan_account_surrkey = las.loan_account_surrkey
    
Left join cmwdw_insurance.vwdw_property_account_dim pad
    on las.property_account_surrkey=pad.property_account_surrkey

Left join ds_ins_risk.ins_risk_last emili
    on emili.lnacctno = lad.cmhc_loan_account_nbr

left join ds_ins_risk.ins_risk_first risk_f
    on lad.cmhc_loan_account_nbr = risk_f.lnacctno
    
Left join cmwdw_insurance.vwdw_borrower_account_basic_dim bor
    on las.first_borrower_account_surrkey = bor.borrower_account_surrkey

Left join cmwdw_insurance.vwdw_lender_financial_trnst_dim lend
    on las.initiation_lender_financial_transit_surrkey = lend.lender_financial_transit_surrkey

where ((emili.aplcapro >='20200101') or (emili.aplcrecd >= '20200101' )) /* Should it only be by received date?*/

    AND lad.low_ratio_mortgage_tcd not in ('01','02')
    /*AND emili.APLMTHCD = '04' removed as not really needed*/
        
"""
ss = spark
df = ss.sql(
    sql.format(
        a.get("CURRENT_YTD_BEG"),
        a.get("CURRENT_YTD_END"),
        a.get("PRIOR_YTD_BEG"),
        a.get("PRIOR_YTD_END"),
        a.get("REPORT_MNTH_6").strftime("%Y%m%d"),
        a.get("REPORT_MNTH_18").strftime("%Y%m%d"),
        a.get("PRIOR_YTD_BEG2"),
        a.get("PRIOR_YTD_END2"),
    )
)

In [0]:
# Perform the join operation
df = df.join(eco_plus, df.cmhc_loan_account_nbr == eco_plus.cmhcno, "left")

# Drop the redundant cmhcno column from eco_plus_selected after the join
df = df.drop(eco_plus.cmhcno)


In [0]:
# Filter the dataframe to display only rows where eco_plus_app_ind is not null
#filtered_df = df.filter(df.eco_plus_app_ind.isNotNull())
#display(filtered_df.limit(10))

In [0]:
#checking how many lenders
# distinct_count = df.select("parent_level1_lender_nm_en").distinct().count()
#print("Distinct count:", distinct_count)

#count_rows=df.count()
#print(count_rows)

In [0]:
spark.conf.set("spark.databricks.io.cache.enabled", "true")

In [0]:
#convert to pandas to apply next step on columns
df=df.toPandas()

In [0]:
#just fyi
#df.display()

In [0]:
#column1_sum = df['finalized'].sum()
#print(column1_sum)

In [0]:
#check for duplicates
#duplicates = df.duplicated(subset=['cmhc_loan_account_nbr']).sum()
#print(duplicates)

In [0]:
#Remove specialt charaters from Contact name and Original Rep fields
#df.replace({"Contact_nm":{"-":"", "'":""}, 'Original_Rep':{"-":"", "'":""}})
df['Original_Rep'] = df['Original_Rep'].replace(["-", "'"], "", regex=True).str.upper()
df['Contact_nm'] = df['Contact_nm'].replace(["-", "'","FAXEMILI"], "", regex=True).str.upper()
#df['Original_Rep'] = df['Original_Rep'].replace(['-', ' '], '', regex=True).str.upper()

In [0]:
# Create 6 new columns with different reordered Contact_nm words to match with BNS Name List
df["Contact_1"] = (
    df["Contact_nm"].astype(str).str.split().str[0].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[1].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[2].fillna("")
)
df["Contact_2"] = (
    df["Contact_nm"].astype(str).str.split().str[0].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[2].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[1].fillna("")
)
df["Contact_3"] = (
    df["Contact_nm"].astype(str).str.split().str[1].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[0].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[2].fillna("")
)
df["Contact_4"] = (
    df["Contact_nm"].astype(str).str.split().str[1].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[2].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[0].fillna("")
)
df["Contact_5"] = (
    df["Contact_nm"].astype(str).str.split().str[2].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[1].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[0].fillna("")
)
df["Contact_6"] = (
    df["Contact_nm"].astype(str).str.split().str[2].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[0].fillna("")
    + df["Contact_nm"].astype(str).str.split().str[1].fillna("")
)

# Create 6 new columns with different reordered Original_Rep words to match with HFS Name List
df["Rep_1"] = (
    df["Original_Rep"].astype(str).str.split().str[0].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[1].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[2].fillna("")
)
df["Rep_2"] = (
    df["Original_Rep"].astype(str).str.split().str[0].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[2].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[1].fillna("")
)
df["Rep_3"] = (
    df["Original_Rep"].astype(str).str.split().str[1].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[0].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[2].fillna("")
)
df["Rep_4"] = (
    df["Original_Rep"].astype(str).str.split().str[1].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[2].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[0].fillna("")
)
df["Rep_5"] = (
    df["Original_Rep"].astype(str).str.split().str[2].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[1].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[0].fillna("")
)
df["Rep_6"] = (
    df["Original_Rep"].astype(str).str.split().str[2].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[0].fillna("")
    + df["Original_Rep"].astype(str).str.split().str[1].fillna("")
)

df_cu_s = ss.createDataFrame(df)
df_cu_s.createOrReplaceTempView("df_cu_s")
# print("Loan Level Base Data with CU Region row count: ", df_cu_s.count())
print(
    "Table creation for Loan Level Base Data with Parent lender and CU Region completed at: ",
    str(datetime.now()),
)

Table creation for Loan Level Base Data with Parent lender and CU Region completed at:  2024-09-06 16:48:26.379542


In [0]:
#df_cu_s.display()

In [0]:
# Read the BNS Name List Excel File and save as dataframe
from pyspark.sql.functions import trim
import pandas as pd

bns = pd.read_excel(
    "BNS Name List Last Update.xlsx", dtype={"INACTIVE_DATE_LASTUPDATE": str}
).astype(str)
bns["LENDER_CHANNEL_LASTUPDATE"] = bns["LENDER_CHANNEL_LASTUPDATE"].replace(
    " ", "", regex=True
)
# bns['EFFECTIVE_DATE_LASTUPDATE'] = pd.to_datetime(bns['EFFECTIVE_DATE_LASTUPDATE'], yearfirst=True)
# bns['INACTIVE_DATE_LASTUPDATE'] = pd.to_datetime(bns['INACTIVE_DATE_LASTUPDATE'], yearfirst=True)
df_bns = ss.createDataFrame(bns)
df_bns = df_bns.withColumn(
    "LENDER_CHANNEL_LASTUPDATE", trim(df_bns.LENDER_CHANNEL_LASTUPDATE)
)
df_bns.createOrReplaceTempView("df_bns")

# Read the HFS Name List Excel File and save as dataframe
hfs = pd.read_excel(
    "HFS Name List Last Update.xlsx",
    dtype={"INACTIVE_DATE_LASTUPDATE": str, "EFFECTIVE_DATE_LASTUPDATE": str},
).astype(str)
# hfs['EFFECTIVE_DATE_LASTUPDATE'] = pd.to_datetime(hfs['EFFECTIVE_DATE_LASTUPDATE'], yearfirst=True)
# hfs['INACTIVE_DATE_LASTUPDATE'] = pd.to_datetime(hfs['INACTIVE_DATE_LASTUPDATE'], yearfirst=True)
df_hfs = ss.createDataFrame(hfs)
df_hfs.createOrReplaceTempView("df_hfs")

In [0]:
# Merge Base Data with BNS Employee Name List with VP, Sales Manager info
sql_a = """
select
t1.*,
t2.EMPLOYEENO_LASTUPDATE as RU_EMPLOYEENO, 
t2.NAME_LASTUPDATE as RU_NAME1, 
t2.LENDER_CHANNEL_LASTUPDATE as bns_lender_channel, 
t2.EFFECTIVE_DATE_LASTUPDATE as RU_Effectivedt, 
t2.INACTIVE_DATE_LASTUPDATE as RU_Inactivedt, 
t2.REGIONAL_VP_LASTUPDATE as RU_VP, 
t2.PROVINCE_LASTUPDATE as RU_Province, 
    
case when t2.EMPLOYEENO_LASTUPDATE is null then 0 else 1 
  end as bnsnamelistmatch_ind,

    
CASE
WHEN t1.financial_transit IN ('002-01339', '002-13151', '002-27912','002-78386') THEN 'MOSE'
WHEN t1.financial_transit IN ('002-85712') THEN
                  CASE WHEN business_source_en = 'BROKER' THEN 'MOSE' 
                       WHEN business_source_en = 'MORTGAGE SPECIALIST' AND t2.LENDER_CHANNEL_LASTUPDATE = 'MOSE' THEN 'MOSE' 
                  ELSE 'HFLM' 
                  END
    WHEN t1.financial_transit IN ('002-77966') THEN 
        CASE WHEN business_source_en = 'MORTGAGE SPECIALIST' THEN 'MOSE'
             WHEN business_source_en NOT IN ('MORTGAGE SPECIALIST') THEN 
                 CASE WHEN t2.LENDER_CHANNEL_LASTUPDATE = 'BRM' THEN 'BRM'
                 ELSE 'MOSE'
                 END
        END
    WHEN t1.financial_transit NOT IN ('002-77966','002-85712','002-01339', '002-13151', '002-27912','002-78386') THEN 'BRANCH'
    END AS lenderchannel
    
from df_cu_s t1 
left join df_bns t2 
on 
((trim(t1.Contact_1) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_1) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_2) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_2) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_3) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_3) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Contact_4) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_4) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Contact_5) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_5) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Contact_6) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Contact_6) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
"""

df_base_bns = ss.sql(sql_a)
df_base_bns.createOrReplaceTempView("df_base_bns")
# print("Merged with BNS Employee List table row count: ", df_base_bns.count())
print(
    "Merged with BNS Employee List Dataframe creation completed at: ",
    str(datetime.now()),
)

Merged with BNS Employee List Dataframe creation completed at:  2024-09-06 16:48:27.474329


In [0]:
#Fill NULL HFS EFFECTIVE_DATE_LASTUPDATE column with default time
df_hfs2 = df_hfs.fillna({'EFFECTIVE_DATE_LASTUPDATE':'1900-01-01 00:00:00'})
df_hfs2.createOrReplaceTempView("df_hfs2")

In [0]:
#Create Dataframe for HFS VP mapping
sql_hfs = """
select distinct *
from df_hfs2 
where ROLE_LASTUPDATE = "SMS"
"""

df_hfs_map = ss.sql(sql_hfs)
df_hfs_map.createOrReplaceTempView("df_hfs_map")

In [0]:
# Merge Previous Dataframe with HFS home finance sales Employee Name List with VP, Sales Manager info.
sql_b = """
select 
t1.cmhc_loan_account_nbr,
    t2.EMPLOYEENO_LASTUPDATE as HFS_EMPLOYEENO, 
    t2.NAME_LASTUPDATE as HFS_NAME1, 
    t2.NAME2_LASTUPDATE as HFS_NAME2,
    t2.ROLE_LASTUPDATE as HFS_ROLE, 
    t2.EFFECTIVE_DATE_LASTUPDATE as HFS_Effectivedt, 
    t2.INACTIVE_DATE_LASTUPDATE as HFS_Inactivedt, 
    t2.SUPERVISOR_LASTUPDATE as HFS_Supervisor, 
    t2.PROVINCE_LASTUPDATE as HFS_Province,  
    t2.ROLE_LASTUPDATE, 
    t3.SUPERVISOR_LASTUPDATE as HFS_VP,
    
case when t2.NAME_LASTUPDATE is not null then 1 else 0
end as hfsnamematch_ind,

case when t3.NAME_LASTUPDATE is not null then 1 else 0
end as vphfsnamematch_ind

from df_cu_s t1
left join df_hfs2 t2
on
((trim(t1.Rep_1) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_1) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_2) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_2) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_3) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_3) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
or 
((trim(t1.Rep_4) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_4) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Rep_5) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_5) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) or 
((trim(t1.Rep_6) = trim(t2.NAME_LASTUPDATE) or (T2.NAME2_LASTUPDATE IS NOT NULL AND trim(t1.Rep_6) = trim(t2.NAME2_LASTUPDATE)))
and (t1.aplcrecd >= t2.effective_date_lastupdate and (t1.aplcrecd < t2.INACTIVE_DATE_LASTUPDATE OR t2.inactive_date_lastupdate is null))
) 
left join df_hfs_map t3  
ON (trim(t2.SUPERVISOR_LASTUPDATE) = trim(t3.NAME_LASTUPDATE) or 
(trim(T3.NAME2_LASTUPDATE) IS NOT NULL AND trim(t2.SUPERVISOR_LASTUPDATE) = trim(t3.NAME2_LASTUPDATE))) 
AND
(t1.aplcrecd >= t3.effective_date_lastupdate and 
(t1.aplcrecd < t3.INACTIVE_DATE_LASTUPDATE OR t3.inactive_date_lastupdate is NULL))
"""

df_base_hfs = ss.sql(sql_b)
df_base_hfs.createOrReplaceTempView("df_base_hfs")
# print("Merged with HFS Employee List table row count: ", df_base_hfs.count())
print(
    "Merged with HFS Employee List Dataframe creation completed at: ",
    str(datetime.now()),
)

Merged with HFS Employee List Dataframe creation completed at:  2024-09-06 16:48:27.847607


In [0]:
#Merge BNS and HFS Name List together
sql_c = """
select t1.*,
    t2.HFS_EMPLOYEENO, 
    t2.HFS_NAME1, 
    t2.HFS_NAME2,
    t2.HFS_ROLE, 
    t2.HFS_Effectivedt, 
    t2.HFS_Inactivedt, 
    t2.HFS_Supervisor, 
    t2.HFS_Province,  
    t2.ROLE_LASTUPDATE as HFS_ROLE_LASTUPDATE, 
    t2.HFS_VP
from df_base_bns t1
left join df_base_hfs t2
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr
"""

df_base_all = ss.sql(sql_c)
df_base_all.createOrReplaceTempView("df_base_all")
print("Base table with BNS and HFS Employee List table row count: ", df_base_all.count())
print("Base Table with BNS and HFS Employee List dataframe creation completed at: ", str(datetime.now()))

Base table with BNS and HFS Employee List table row count:  430669
Base Table with BNS and HFS Employee List dataframe creation completed at:  2024-09-06 16:48:32.700052


In [0]:
# Calculate Fiscal Year/Month for receive and ever approve date
sql_fs = """
select distinct 
cmhc_loan_account_nbr,
aplcapro,
aplcrecd 
from df_base_all
"""

df_fiscal = ss.sql(sql_fs).toPandas()
df_fiscal["aplcrecd"] = pd.to_datetime(df_fiscal["aplcrecd"])
df_fiscal["aplcapro"] = pd.to_datetime(df_fiscal["aplcapro"])
df_fiscal["Rec_fiscal_year"] = df_fiscal["aplcrecd"].dt.to_period("Q-OCT").dt.qyear
df_fiscal["Ever_fiscal_year"] = df_fiscal["aplcapro"].dt.to_period("Q-OCT").dt.qyear

# Merge Fiscal Year with Main Data and save table in sandbox
df_fiscal_ss = ss.createDataFrame(df_fiscal)
df_fiscal_ss.createOrReplaceTempView("df_fiscal")

sql_fs2 = """
select t1.*,
t2.Rec_fiscal_year,
t2.Ever_fiscal_year
from df_base_all t1 
left join df_fiscal t2
on t1.cmhc_loan_account_nbr = t2.cmhc_loan_account_nbr
"""

df_fiscal2 = ss.sql(sql_fs2)

In [0]:
ss.conf.set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")

In [0]:
df_fiscal2.write.mode("overwrite").format("orc").saveAsTable("sb_rmo_ho_reporting.test_lender_dashboard")
#print("Final Base Table with Channel, Source, BNS Contact row count: ", df_fiscal2.count())
print("Final Base Table with Channel, Source, BNS Contact creation completed at: ", str(datetime.now()))

Final Base Table with Channel, Source, BNS Contact creation completed at:  2024-09-06 16:49:10.541347


In [0]:
#distinct_count = df_fiscal2.select("parent_level1_lender_nm_en").distinct().count()
#print("Distinct count:", distinct_count)